# Building a regression model for predicting house sale prices

In [193]:
import pickle
import pathlib

import numpy as np
import pandas as pd

In [194]:
DATA_DIR = pathlib.Path.cwd().parent / 'data'
print(DATA_DIR)

c:\Users\code\Desktop\ML\BANGU2\ames-Bangu1\data


In [195]:
clean_data_path = DATA_DIR / 'processed' / 'ames_clean.pkl'

In [196]:
with open(clean_data_path, 'rb') as file:
    data = pickle.load(file)

In [197]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2872 entries, 0 to 2929
Data columns (total 73 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   MS.SubClass                 2872 non-null   category
 1   MS.Zoning                   2872 non-null   category
 2   Lot.Frontage                2872 non-null   float64 
 3   Lot.Area                    2872 non-null   float64 
 4   Lot.Shape                   2872 non-null   category
 5   Land.Contour                2872 non-null   category
 6   Lot.Config                  2872 non-null   category
 7   Land.Slope                  2872 non-null   category
 8   Neighborhood                2872 non-null   category
 9   Bldg.Type                   2872 non-null   category
 10  House.Style                 2872 non-null   category
 11  Overall.Qual                2872 non-null   category
 12  Overall.Cond                2872 non-null   category
 13  Roof.Style             

In [198]:
model_data = data.copy()

## Preparing the data for the model

Now that we have the data all cleaned, and all the missing values accounted for, lets focus on transforming the data for the model.

Lets remember what a model is. 

- A predictive model is a **set** of functions that receive data as input and produce a prediction, that is, an estimate of the target value as output.
- **Train** a model is to search the set of candidate functions for one that adequately represents the **training dataset**.
- The adequacy of a candidate function to the training data is usually determined by a **loss function** that measures how well the predictions of the function match the real values of the target within the training dataset. It is common to define a *loss function per data item* (e.g. absolute error, quadratic error, etc) and to construct the *loss function over the dataset* as the *average prediction loss*.

Many models are **parametric models**. In this case, each function of the set of functions that makes the model is constructed from a vector of **parameters** that define the function, forming a **parametric function**. For instance: the linear model constructs prediction values out of a linear combination of the input features, plus a constant. The weights of the linear combination plus the constant are the parameters of the model. The set of functions that can be represented by this model is given by all possible values of the vector of parameters that define the function.

Some models are called **non-parametric models**. These models usually do not have a parametric form (like the linear model). But the terminology is a bit misleading, though: usually these models *do* have parameters, and potentially an open-ended set of them! For instance, consider the "decision tree" model, which is one of the most prominent models of this category. The decision tree may not have a formula for the predicted value, but it does have parameters, many of them: each decision in the tree involves a choice of feature and a threshold level, and those choices must be stored as parameters of the model for use in future predictions.

Each model has specific requirements for the format of the input data. Most of the time, the minimum requirement is that:

- All columns are numeric;
- There are no missing values.

Some models have extra requirements. For example: the support-vector-machines model requires that the input features have comparable standard deviations - having features that have large discrepancies between features in terms of their order of magnitude (such as a feature in the fractions of unit range and another in the tens of thousands) will result in poor prediction quality.

And some models may not have any special requirement at all. We will study each of those in detail in this course.

Lets start our study with a simple model: the *multivariate linear regression* model. This is a model that presents the minimum requirements listed above. So we need to do a bit of processing on the original features:

- *Numerical features* stay as given;
- *Categorical features* have to be transformed into numerical features. In order to do so we need to **encode** these features, that is: to transform them into new features that convey the same information, but in a numerical form, and in a way that "makes sense" - we'll see it below.
- *Ordinal features* can be transformed into numerical features in the same way as the caegorical features, or could be assigned increasing numbers in conformity with the ordered nature of the categories of the feature.

## Encoding categorical variables

Lets identify all categorical variables - both nominal (that is, categoricals without category order) and ordinal.

In [199]:
categorical_columns = []
ordinal_columns = []
for col in model_data.select_dtypes('category').columns:
    if model_data[col].cat.ordered:
        ordinal_columns.append(col)
    else:
        categorical_columns.append(col)

In [200]:
ordinal_columns

['Lot.Shape',
 'Land.Slope',
 'Overall.Qual',
 'Overall.Cond',
 'Exter.Qual',
 'Exter.Cond',
 'Heating.QC',
 'Electrical',
 'Kitchen.Qual',
 'Functional',
 'Paved.Drive',
 'Fence']

In [201]:
categorical_columns

['MS.SubClass',
 'MS.Zoning',
 'Land.Contour',
 'Lot.Config',
 'Neighborhood',
 'Bldg.Type',
 'House.Style',
 'Roof.Style',
 'Mas.Vnr.Type',
 'Foundation',
 'Bsmt.Qual',
 'Bsmt.Cond',
 'Bsmt.Exposure',
 'BsmtFin.Type.1',
 'BsmtFin.Type.2',
 'Central.Air',
 'Garage.Type',
 'Garage.Finish',
 'Sale.Type',
 'Sale.Condition',
 'Condition',
 'Exterior']

### Encoding ordinal variables 

Ordinal variables can be transformed into integer numbers in a straightforward manner: the lowest category is assigned the value "zero", the next category is given the value "one", etc. The `Pandas` library has a function for this task: `factorize()`:

In [202]:
for col in ordinal_columns:
    codes, _ = pd.factorize(data[col], sort=True)
    model_data[col] = codes

Lets confirm that the variables are no longer ordinal, but now are integers:

In [203]:
model_data[ordinal_columns].info()

<class 'pandas.core.frame.DataFrame'>
Index: 2872 entries, 0 to 2929
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   Lot.Shape     2872 non-null   int64
 1   Land.Slope    2872 non-null   int64
 2   Overall.Qual  2872 non-null   int64
 3   Overall.Cond  2872 non-null   int64
 4   Exter.Qual    2872 non-null   int64
 5   Exter.Cond    2872 non-null   int64
 6   Heating.QC    2872 non-null   int64
 7   Electrical    2872 non-null   int64
 8   Kitchen.Qual  2872 non-null   int64
 9   Functional    2872 non-null   int64
 10  Paved.Drive   2872 non-null   int64
 11  Fence         2872 non-null   int64
dtypes: int64(12)
memory usage: 291.7 KB


Compare the original values with the encoded values:

In [204]:
data['Lot.Shape'].value_counts()

Lot.Shape
Reg    1825
IR1     956
IR2      76
IR3      15
Name: count, dtype: int64

In [205]:
model_data['Lot.Shape'].value_counts()

Lot.Shape
0    1825
1     956
2      76
3      15
Name: count, dtype: int64

### Encoding nominal variables

With nominal variables there is no notion of order among categories. Therefore, it would be a conceptual mistake to encode them in the same manner as the ordinal variables. For instance, consider the `Exterior` variable:

In [206]:
model_data['Exterior'].value_counts()

Exterior
VinylSd    1024
HdBoard     438
MetalSd     432
Wd Sdng     400
Plywood     218
CemntBd     124
BrkFace      86
WdShing      55
AsbShng      41
Stucco       41
Other        13
Name: count, dtype: int64

We cannot assign an order here, lest we end up with equations like `HdBoard` + `Plywood` = `CemntBd`, which are nonsense. 

The strategy here to encode `Exterior` is to create several new numerical variables to represent the membership of a given data item to one of the `Exterior` categories. These are called **dummy variables**. Each of these new variables contain only the values "zero" or "one" (i.e. they are binary variables), where $1$ denotes that the data item belongs to the category represented by the variable. Evidently, for a given data item, only one dummy variable has a value of $1$, all remaining are $0$.

There are two types of dummy variable encoding:

- "One-hot" encoding: in this case we create one dummy variable per category. Let's look at the `Exterior` feature as an example. The `Pandas` function `get_dummies()` can do the encoding for us:

In [207]:
original_data = model_data['Exterior']
encoded_data = pd.get_dummies(original_data)

aux_dataframe = encoded_data
aux_dataframe['Exterior'] = original_data.copy()

aux_dataframe.head().transpose()

,0,1,2,3,4
AsbShng,False,False,False,False,False
BrkFace,True,False,False,True,False
CemntBd,False,False,False,False,False
HdBoard,False,False,False,False,False
MetalSd,False,False,False,False,False
Plywood,False,False,False,False,False
Stucco,False,False,False,False,False
VinylSd,False,True,False,False,True
Wd Sdng,False,False,True,False,False
WdShing,False,False,False,False,False


Observe that for each value of `Exterior`, only the corresponding dummy is flagged.

One-hot encoding is a popular technique in Machine Learning. Statisticians, however, prefer a slightly different way of dummy encoding which is:

- Choose a category to *not encode* (this is called the *base category*)
- Generate dummies for the remaining categories. That is:
    - If the data item belongs to the base category, no dummy receives a value of $1$;
    - Otherwise, set the corresponding dummy to $1$.

The same `get_dummies()` function of `Pandas` can do this automatically with the `drop_first` argument:

In [208]:
original_data = model_data['Exterior']
encoded_data = pd.get_dummies(original_data, drop_first=True)

aux_dataframe = encoded_data
aux_dataframe['Exterior'] = original_data.copy()

aux_dataframe.head().transpose()

,0,1,2,3,4
BrkFace,True,False,False,True,False
CemntBd,False,False,False,False,False
HdBoard,False,False,False,False,False
MetalSd,False,False,False,False,False
Plywood,False,False,False,False,False
Stucco,False,False,False,False,False
VinylSd,False,True,False,False,True
Wd Sdng,False,False,True,False,False
WdShing,False,False,False,False,False
Other,False,False,False,False,False


Notice that we are now missing the dummy variable for the `AsbShng` category.

Why to encode things this way? If we don't drop one of the dummies, then it will always be the case that the sum of the values of the dummies is $1$ (since each data item must belong to one of the categories). The linear model, particularly very popular with the statisticians, implies the existence of a fictitious feature containing, for all data items, the value $1$. Hence we end up having a set of variables where a linear combination of them (in this case, the sum of the dummies) matches the value at another variable. This has numerical computing implications for the linear model, that we will discuss in class.

Since we want to use the linear model in this notebook, lets encode all categoricals with the `drop_first` alternative.

In [209]:
model_data = pd.get_dummies(model_data, drop_first=True)

Now our dataset has a lot more variables!

In [210]:
model_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2872 entries, 0 to 2929
Columns: 163 entries, Lot.Frontage to Exterior_Other
dtypes: bool(114), float64(37), int64(12)
memory usage: 1.4 MB


In [211]:
for cat in categorical_columns:
    dummies = []
    for col in model_data.columns:
        if col.startswith(cat + "_"):
            dummies.append(f'"{col}"')
    dummies_str = ', '.join(dummies)
    print(f'From column "{cat}" we made {dummies_str}\n')

From column "MS.SubClass" we made "MS.SubClass_30", "MS.SubClass_50", "MS.SubClass_60", "MS.SubClass_70", "MS.SubClass_80", "MS.SubClass_85", "MS.SubClass_90", "MS.SubClass_120", "MS.SubClass_160", "MS.SubClass_190", "MS.SubClass_Other"

From column "MS.Zoning" we made "MS.Zoning_RH", "MS.Zoning_RL", "MS.Zoning_RM"

From column "Land.Contour" we made "Land.Contour_HLS", "Land.Contour_Low", "Land.Contour_Lvl"

From column "Lot.Config" we made "Lot.Config_CulDSac", "Lot.Config_Inside", "Lot.Config_Others"

From column "Neighborhood" we made "Neighborhood_BrDale", "Neighborhood_BrkSide", "Neighborhood_ClearCr", "Neighborhood_CollgCr", "Neighborhood_Crawfor", "Neighborhood_Edwards", "Neighborhood_Gilbert", "Neighborhood_IDOTRR", "Neighborhood_MeadowV", "Neighborhood_Mitchel", "Neighborhood_NAmes", "Neighborhood_NPkVill", "Neighborhood_NWAmes", "Neighborhood_NoRidge", "Neighborhood_NridgHt", "Neighborhood_OldTown", "Neighborhood_SWISU", "Neighborhood_Sawyer", "Neighborhood_SawyerW", "Neighb

## Train-test splitting

The data will now be organized as follows:

- The features form a matrix $X$ of size $m \times n$, where $m$ is the number of data items, and $n$ is the number of features.
- The target forms a column-matrix $y$ of length $m$.

In [212]:
X = model_data.drop(columns=['SalePrice']).copy()
y = model_data['SalePrice'].copy()

In [213]:
X.values, y.values

(array([[141.0, 31770.0, 1, ..., False, False, False],
        [80.0, 11622.0, 0, ..., False, False, False],
        [81.0, 14267.0, 1, ..., True, False, False],
        ...,
        [62.0, 10441.0, 0, ..., False, False, False],
        [77.0, 10010.0, 0, ..., False, False, False],
        [74.0, 9627.0, 0, ..., False, False, False]], dtype=object),
 array([5.33243846, 5.0211893 , 5.23552845, ..., 5.12057393, 5.23044892,
        5.27415785]))

This is the typical set-up of a machine learning project. Now we want to train our model *and* verify that the model provides good predictions for *unseen* data items. Why the emphasis on "unseen"? Because there is no use for a model that only gives predictions for the items in the data used to train it - we want our models to *generalize*.

The way to assess the model's performance for unseen values is to split the dataset into two subsets: the **training** and **test** datasets.

We have been using a lot of `Pandas` to manipulate our data so far. From now on we will switch to another very popular library for machine learning in Python: `Scikit-Learn`.

The function `train_test_split()` will take as arguments the dataset to be split, the specification of the fraction of the dataset to be reserved for testing, and a random seed value - so that the split will always be the same whenever we run our notebook. This is a customary measure to ensure reproducibility of the notebook.

In [214]:
from sklearn.model_selection import train_test_split

In [215]:
RANDOM_SEED = 42  # Any number here, really.

In [216]:
Xtrain, Xtest, ytrain, ytest = train_test_split(
    X,
    y,
    test_size=0.25,
    random_state=RANDOM_SEED,
)


In [217]:
X.shape, Xtrain.shape, Xtest.shape

((2872, 162), (2154, 162), (718, 162))

In [218]:
y.shape, ytrain.shape, ytest.shape

((2872,), (2154,), (718,))

## Fitting a linear model

Lets start with fitting a linear model for regression. The linear model is one of the oldest and most used models for regression, due to its simplicity and strong statistical roots. A proper statistical approach to the understanding of the linear model consists of:

- Understanding the statistical premises of the linear model;
- Analyzing the features to verify that the preliminary conditions of this modeling strategy are satisfied;
- Fitting the model;
- Analyzing the residuals to confirm that the post-fit conditions are satisfied.

Lets discuss these topics in more detail:

### The statistical approach to the linear model

In machine learning we are more interested in the predictive capability of a model, rather than its inductive use to analyze the relation between features and target (or independent and dependent variables, in the statistical terminology). But even to a machine learning practitioner, understanding the statistical basis of the linear model may lead to better predictive performance. For instance:

- Having a symmetrical residual is usually associated with better mean-squared-error (MSE) than having a long-tailed assymmetric residual;
- Non-significant parameters (in a hypothesis-testing sense) may indicate superfluous variables in the model, which could be associated with reduced performance in the test dataset (i.e. poor generalization).

So what is the linear model in statistics? A statistical model is a way to describe probabilistically the relation between features and targets, usually in a parametric way.

Mathematically, we are searching for a *conditional probability* density model of the form $f(Y = y | \mathbf{x}, \theta)$ where $\mathbf{x}$ is the feature vector, $Y$ is the random variable associated with the target, and $\theta$ is the vector of parameters. In plain language, we would like to describe the probability distribution of the target variable when the value of the feature vector is known and the parameters of the model are given.

In the linear model, we postulate that the data $y$ is generated from the feature vector $\mathbf{x}$ plus a random Gaussian noise of fixed standard deviation, as shown in the equation below:

$$
y = \theta_0 + \theta_1 x_1 + \theta_2 x_2 + \cdots + \theta_n x_n + \varepsilon
$$

where $\varepsilon \sim N(0, \sigma)$

The addition of the noise term causes the $y$ value to become a random variable itself, thus making the probabilistic model mentioned above. For a given value of $\mathbf{x}$, the value of $y$ is obtained by adding the constant value $\theta_0 + \theta_1 x_1 + \theta_2 x_2 + \cdots + \theta_n x_n$ to a normal random variable $\varepsilon$. Remember that, for normal random variables, adding a constant keeps the variable normal, with a shifted mean-value parameter. Therefore:

$$
Y \sim N(\mu = (\theta_0 + \theta_1 x_1 + \theta_2 x_2 + \cdots + \theta_n x_n), \sigma)
$$

Lets write

$$
\hat{y} = \theta_0 + \theta_1 x_1 + \theta_2 x_2 + \cdots + \theta_n x_n
$$

for simplicity, then the model above is rewritten as:

$$
Y \sim N(\mu = \hat{y}, \sigma)
$$

When we have a dataset $D = \{(\mathbf{x}_1, y_1), (\mathbf{x}_2, y_2), \cdots, (\mathbf{x}_m, y_m)\}$ of several $(\mathbf{x}, y)$ pairs, what is their joint conditional probability density function $f(y_1, y_2, \cdots, y_m | x_1, x_2, \cdots, x_n, \theta)$?

We will make another supposition of the linear model: that the $(\mathbf{x}, y)$ examples were obtained independently, and that the value of one does not impact the probability of the other. Therefore:

$$
f(y_1, y_2, \cdots, y_m | x_1, x_2, \cdots, x_n, \theta) = f(y_1| x_1, \theta) f(y_2| x_2, \theta) \cdots f(y_m| x_m, \theta) 
$$

Remember that the normal probability density function is as follows:

$$
Y \sim N(\mu, \sigma) \Rightarrow f(y) = \frac{1}{\sigma \sqrt{2 \pi}} \exp \left(-\frac{1}{2}\frac{(y - \mu)^2}{\sigma^2} \right)
$$

Thus:

$$
Y \sim N(\mu = \hat{y}, \sigma) \Rightarrow f(y) = \frac{1}{\sigma \sqrt{2 \pi}} \exp \left(-\frac{1}{2}\frac{(y - \hat{y})^2}{\sigma^2} \right)
$$

And the joint conditional probability density function of the entire dataset becomes:


$$
f(y_1, y_2, \cdots, y_m | x_1, x_2, \cdots, x_n, \theta) = \prod_{i=1}^{m} \left(\frac{1}{\sigma \sqrt{2 \pi}} \exp \left(-\frac{1}{2}\frac{(y_i - \hat{y_i})^2}{\sigma^2} \right) \right)
$$

Expanding the product we have:

$$
\begin{align*}
f(y_1, y_2, \cdots, y_m | x_1, x_2, \cdots, x_n, \theta) & = & \prod_{i=1}^{m} \left(\frac{1}{\sigma \sqrt{2 \pi}} \exp \left(-\frac{1}{2}\frac{(y_i - \hat{y_i})^2}{\sigma^2} \right) \right) \\
& = & \prod_{i=1}^{m} \left(\frac{1}{\sigma \sqrt{2 \pi}} \right) \prod_{i=1}^{m} \left( \exp \left(-\frac{1}{2}\frac{(y_i - \hat{y_i})^2}{\sigma^2} \right) \right) \\
& = & \left(\frac{1}{\sigma \sqrt{2 \pi}} \right)^{m} \exp \left(\sum_{i=1}^{m} \left(-\frac{1}{2}\frac{(y_i - \hat{y_i})^2}{\sigma^2} \right) \right) \\
& = & \left(\frac{1}{\sigma \sqrt{2 \pi}} \right)^{m} \exp \left(- \frac{1}{2 \sigma} \sum_{i=1}^{m} (y_i - \hat{y_i})^2 \right)
\end{align*}
$$

What are the "best" value for the parameters of the linear model? We can search for the parameters that maximize the joint conditional probability density function of the dataset. This function is called the *likelihood* of the parameters, and therefore our solution here is called a *"maximum likelihood estimate"* of the parameters.

So we are looking for a value $\theta^{\star}$ of $\theta$ to maximize $f(y_1, y_2, \cdots, y_m | x_1, x_2, \cdots, x_n, \theta)$, that is:

$$
\begin{align*}
\theta^{\star} & = & \argmax_{\theta} \left\{ f(y_1, y_2, \cdots, y_m | x_1, x_2, \cdots, x_n, \theta) \right\}\\
& = & \argmax_{\theta} \left\{ \left(\frac{1}{\sigma \sqrt{2 \pi}} \right)^{m} \exp \left(- \frac{1}{2 \sigma} \sum_{i=1}^{m} (y_i - \hat{y_i})^2 \right) \right\} \\
& = & \argmax_{\theta} \left\{ \exp \left(- \frac{1}{2 \sigma} \sum_{i=1}^{m} (y_i - \hat{y_i})^2 \right) \right\} \\
& = & \argmax_{\theta} \left\{ - \frac{1}{2 \sigma} \sum_{i=1}^{m} (y_i - \hat{y_i})^2 \right\} \\
& = & \argmin_{\theta} \left\{ \frac{1}{2 \sigma} \sum_{i=1}^{m} (y_i - \hat{y_i})^2 \right\} \\
& = & \argmin_{\theta} \left\{ \sum_{i=1}^{m} (y_i - \hat{y_i})^2 \right\} \\
& = & \argmin_{\theta} \left\{ \frac{1}{m} \sum_{i=1}^{m} (y_i - \hat{y_i})^2 \right\} \\
\end{align*}
$$

Hey, look who we found! Our old friend MSE (mean-squared-error)!

So, in the end, here are the lessons:

- The statistical formulation of the linear model leads to the same error formulation of machine learning, which only cares for the prediction quality.
- The statistical linear model has several assumptions:
    - The samples are independent;
    - The target is *truly* generated from the linear predictive formula plus a normally-distributed error;
    - The error has zero mean and constant standard deviation (the *homoscedasticity* hypothesis);
    - There is no error in the feature measurement. That is, $\mathbf{x}_{i}$ are constants, not random variables. All the error is in the target;
- If the assumptions of the linear model are satisfied, you can analyze the parameters with greater sophistication (the machine learning formulation does not bring this finesse). For instance, you can run hypothesis tests on the values of the parameters to determine whether they refute the null hypothesis $\theta_i = 0$ with a given statistical significance level. Which, in plain language, means that you don't really trust that the associated feature impacts the target, or if it is just an accident.

## Teste Inicial - 

Modelos - 
- `LinearRegression`
- `Ridge`
- `Lasso`
- `ElasticNet`

In [237]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet

In [238]:
# Teste com todos os modelos

linreg = LinearRegression()
ridge = Ridge()
lasso = Lasso()
elasticNet = ElasticNet()

linreg.fit(Xtrain, ytrain)
ridge.fit(Xtrain, ytrain)
lasso.fit(Xtrain, ytrain)
elasticNet.fit(Xtrain, ytrain)

ElasticNet()

In [239]:
pred_linreg = linreg.predict(Xtest)
pred_ridge = ridge.predict(Xtest)
pred_lasso = lasso.predict(Xtest)
pred_elasticNet = elasticNet.predict(Xtest)

In [240]:
from sklearn.metrics import mean_squared_error

RMSE_lin = np.sqrt(mean_squared_error(ytest, pred_linreg))
RMSE_ridge = np.sqrt(mean_squared_error(ytest, pred_ridge))
RMSE_lasso = np.sqrt(mean_squared_error(ytest, pred_lasso))
RMSE_elasticNet = np.sqrt(mean_squared_error(ytest, pred_elasticNet))


In [241]:
error_percent = 100 * (10**RMSE_lin - 1)
print(f'Average error for LinearRegression is {error_percent:.2f}%')

error_percent = 100 * (10**RMSE_ridge - 1)
print(f'Average error for Ridge is {error_percent:.2f}%')

error_percent = 100 * (10**RMSE_lasso - 1)
print(f'Average error for Lasso is {error_percent:.2f}%')

error_percent = 100 * (10**RMSE_elasticNet - 1)
print(f'Average error for ElasticNet is {error_percent:.2f}%')

Average error for LinearRegression is 11.86%
Average error for Ridge is 11.83%
Average error for Lasso is 22.81%
Average error for ElasticNet is 19.92%


In [242]:
import joblib

# Seu modelo treinado
modelo_treinado = ridge

joblib.dump(modelo_treinado, 'modelo_treinado.pkl')


['modelo_treinado.pkl']

In [224]:
#Make a grid search with Ridge
from sklearn.model_selection import GridSearchCV

# código utilizado: https://stackoverflow.com/questions/57376860/how-to-run-gridsearchcv-with-ridge-regression-in-sklearn

parameters = {'alpha':[0.0001,0.001,0.005,0.01,0.05,0.1,0.5,1,2,5,10,11,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,92,93,94,95,96,97,98,99,100]}

model = Ridge()

ridge_reg= GridSearchCV(model, parameters, scoring='neg_mean_squared_error',cv=5,n_jobs=-1)

ridge_reg.fit(Xtrain,ytrain)

best_model = ridge_reg.best_estimator_


pred_pipeRidge = best_model.predict(Xtest)

from sklearn.metrics import mean_squared_error

RMSE_pipeRidge = np.sqrt(mean_squared_error(ytest, pred_pipeRidge))

error_percent = 100 * (10**RMSE_pipeRidge - 1)
print(f'Average error for Ridge Pipeline is {error_percent:.2f}%')


Average error for Ridge Pipeline is 11.87%


## Teste - Pipelines

Utilizar o StandardScaler e PolynomialFeatures antes de fazer o fit e predict do modelo.

Modelos - 
- `LinearRegression`
- `Ridge`

In [225]:
from sklearn.discriminant_analysis import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

pipeLin = Pipeline([
    ('scaler', StandardScaler()),
    # ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('cls', LinearRegression())
])

pipeLin.fit(Xtrain,ytrain)
pred_pipeLin = pipeLin.predict(Xtest)


from sklearn.metrics import mean_squared_error

RMSE_pipeLin = np.sqrt(mean_squared_error(ytest, pred_pipeLin))

error_percent = 100 * (10**RMSE_pipeLin - 1)
print(f'Average error for LinearRegression Pipeline is {error_percent:.2f}%')

Average error for LinearRegression Pipeline is inf%


C:\Users\code\AppData\Local\Temp\ipykernel_7948\1893494891.py:19: RuntimeWarning: overflow encountered in scalar power
  error_percent = 100 * (10**RMSE_pipeLin - 1)


In [226]:
from sklearn.discriminant_analysis import StandardScaler
from sklearn.pipeline import Pipeline

pipeRidge = Pipeline([
    ('scaler', StandardScaler()),
    ('cls', Ridge())
])

pipeRidge.fit(Xtrain,ytrain)
pred_pipeRidge = pipeRidge.predict(Xtest)


from sklearn.metrics import mean_squared_error

RMSE_pipeRidge = np.sqrt(mean_squared_error(ytest, pred_pipeRidge))

error_percent = 100 * (10**RMSE_pipeRidge - 1)
print(f'Average error for Ridge Pipeline is {error_percent:.2f}%')

Average error for Ridge Pipeline is 11.85%


## Teste 1
Droppando colunas com poucos dados e/ou correlação com a coluna de target.

Colunas - 'Low.Qual.Fin.SF', 'X2nd.Flr.SF', 'X3Ssn.Porch', 'BsmtFin.SF.2', 'Bsmt.Half.Bath', 'Mas.Vnr.Area'

In [227]:
X = model_data.drop(columns=['SalePrice', 'Low.Qual.Fin.SF', 'X2nd.Flr.SF', 'X3Ssn.Porch', 'BsmtFin.SF.2', 'Bsmt.Half.Bath', 'Mas.Vnr.Area']).copy()
y = model_data['SalePrice'].copy()


In [228]:
from sklearn.model_selection import train_test_split
RANDOM_SEED = 42  # Any number here, really.

Xtrain, Xtest, ytrain, ytest = train_test_split(
    X,
    y,
    test_size=0.25,
    random_state=RANDOM_SEED,
)

Ridge básico

In [229]:
from sklearn.metrics import mean_squared_error

ridge = Ridge()

ridge.fit(Xtrain, ytrain)

pred_ridge = ridge.predict(Xtest)

RMSE_ridge = np.sqrt(mean_squared_error(ytest, pred_ridge))

error_percent = 100 * (10**RMSE_ridge - 1)
print(f'Average error for Ridge is {error_percent:.2f}%')

Average error for Ridge is 11.85%


In [230]:
from sklearn.metrics import mean_squared_error

Lasso = Lasso()

Lasso.fit(Xtrain, ytrain)

pred_Lasso = Lasso.predict(Xtest)

RMSE_Lasso = np.sqrt(mean_squared_error(ytest, pred_Lasso))

error_percent = 100 * (10**RMSE_Lasso - 1)
print(f'Average error for Lasso is {error_percent:.2f}%')

Average error for Lasso is 22.87%


Ridge com Pipeline

In [231]:
from sklearn.discriminant_analysis import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from catboost import CatBoostRegressor


columns_to_standardize = ['Lot.Area', 'X1st.Flr.SF', 'Bsmt.Unf.SF', 'Total.Bsmt.SF', 'Lot.Frontage']

preprocessor = ColumnTransformer(
    transformers=[
        ('std_scaler', StandardScaler(), columns_to_standardize),
    ],
    remainder='passthrough'
)

pipeRidge = Pipeline([
    ('preprocessor', preprocessor),
    ('cls', CatBoostRegressor(n_estimators=10000, learning_rate=0.03, random_state=RANDOM_SEED))
])

pipeRidge.fit(Xtrain,ytrain)
pred_pipeRidge = pipeRidge.predict(Xtest)

from sklearn.metrics import mean_squared_error

RMSE_pipeRidge = np.sqrt(mean_squared_error(ytest, pred_pipeRidge))

error_percent = 100 * (10**RMSE_pipeRidge - 1)
print(f'Average error for Ridge Pipeline is {error_percent:.2f}%')

0:	learn: 0.1658581	total: 8.74ms	remaining: 1m 27s
1:	learn: 0.1625457	total: 18.3ms	remaining: 1m 31s
2:	learn: 0.1594914	total: 30.1ms	remaining: 1m 40s
3:	learn: 0.1564608	total: 41.5ms	remaining: 1m 43s
4:	learn: 0.1533944	total: 51.1ms	remaining: 1m 42s
5:	learn: 0.1504435	total: 59.8ms	remaining: 1m 39s
6:	learn: 0.1473981	total: 68.9ms	remaining: 1m 38s
7:	learn: 0.1445312	total: 77.5ms	remaining: 1m 36s
8:	learn: 0.1416264	total: 84.9ms	remaining: 1m 34s
9:	learn: 0.1389581	total: 91.7ms	remaining: 1m 31s
10:	learn: 0.1363485	total: 98.8ms	remaining: 1m 29s
11:	learn: 0.1338384	total: 106ms	remaining: 1m 28s
12:	learn: 0.1313197	total: 113ms	remaining: 1m 27s
13:	learn: 0.1288913	total: 122ms	remaining: 1m 26s
14:	learn: 0.1266306	total: 130ms	remaining: 1m 26s
15:	learn: 0.1243004	total: 137ms	remaining: 1m 25s
16:	learn: 0.1221751	total: 145ms	remaining: 1m 25s
17:	learn: 0.1200139	total: 153ms	remaining: 1m 25s
18:	learn: 0.1178847	total: 162ms	remaining: 1m 24s
19:	learn: 

843:	learn: 0.0253483	total: 6.7s	remaining: 1m 12s
844:	learn: 0.0253343	total: 6.71s	remaining: 1m 12s
845:	learn: 0.0253204	total: 6.72s	remaining: 1m 12s
846:	learn: 0.0252949	total: 6.73s	remaining: 1m 12s
847:	learn: 0.0252727	total: 6.74s	remaining: 1m 12s
848:	learn: 0.0252565	total: 6.75s	remaining: 1m 12s
849:	learn: 0.0252407	total: 6.76s	remaining: 1m 12s
850:	learn: 0.0252274	total: 6.77s	remaining: 1m 12s
851:	learn: 0.0252033	total: 6.77s	remaining: 1m 12s
852:	learn: 0.0251915	total: 6.78s	remaining: 1m 12s
853:	learn: 0.0251771	total: 6.79s	remaining: 1m 12s
854:	learn: 0.0251584	total: 6.8s	remaining: 1m 12s
855:	learn: 0.0251389	total: 6.81s	remaining: 1m 12s
856:	learn: 0.0251209	total: 6.82s	remaining: 1m 12s
857:	learn: 0.0251044	total: 6.82s	remaining: 1m 12s
858:	learn: 0.0250900	total: 6.83s	remaining: 1m 12s
859:	learn: 0.0250703	total: 6.84s	remaining: 1m 12s
860:	learn: 0.0250564	total: 6.85s	remaining: 1m 12s
861:	learn: 0.0250461	total: 6.87s	remaining: 1m

## Teste 2 - Análise dos Dados

Vendo colunas numéricas que precisam de StandardScaler, e apenas selecionando algumas colunas para o PolynomialFeatures.

In [232]:
X = model_data.drop(columns=['SalePrice']).copy()
y = model_data['SalePrice'].copy()

numerical_data = X.select_dtypes('number').copy()
colunas_numericas = list(numerical_data.columns)

from sklearn.model_selection import train_test_split
RANDOM_SEED = 42  # Any number here, really.

Xtrain, Xtest, ytrain, ytest = train_test_split(
    X,
    y,
    test_size=0.25,
    random_state=RANDOM_SEED,
)

colunas_numericas

['Lot.Frontage',
 'Lot.Area',
 'Lot.Shape',
 'Land.Slope',
 'Overall.Qual',
 'Overall.Cond',
 'Mas.Vnr.Area',
 'Exter.Qual',
 'Exter.Cond',
 'BsmtFin.SF.1',
 'BsmtFin.SF.2',
 'Bsmt.Unf.SF',
 'Total.Bsmt.SF',
 'Heating.QC',
 'Electrical',
 'X1st.Flr.SF',
 'X2nd.Flr.SF',
 'Low.Qual.Fin.SF',
 'Gr.Liv.Area',
 'Bsmt.Full.Bath',
 'Bsmt.Half.Bath',
 'Full.Bath',
 'Half.Bath',
 'Bedroom.AbvGr',
 'Kitchen.AbvGr',
 'Kitchen.Qual',
 'TotRms.AbvGrd',
 'Functional',
 'Fireplaces',
 'Paved.Drive',
 'Wood.Deck.SF',
 'Open.Porch.SF',
 'Enclosed.Porch',
 'X3Ssn.Porch',
 'Screen.Porch',
 'Pool.Area',
 'Fence',
 'Misc.Val',
 'Mo.Sold',
 'Yr.Sold',
 'Garage.Age',
 'Remod.Age',
 'House.Age',
 'TotalBath',
 'AllSF',
 'AllFlrsSF',
 'All.Porch.SF',
 'garage_area_by_garage_cars']

lista de colunas numéricas que serão utilizada para fazer o PolynomialFeatures

In [233]:
colunas_numericas_poly = ['Lot.Frontage',
 'Lot.Area',#
 'BsmtFin.SF.1',#
 'Bsmt.Unf.SF',#
 'Total.Bsmt.SF',#
 'X1st.Flr.SF',#
 'Gr.Liv.Area',#
 'Garage.Area',#
 'House.Age']#

Criação de novas colunas

In [234]:
def teste (X):
    df = X.copy()

    df['Has.Screen.Porch'] = (df['Screen.Porch'] > 0).astype(int)

    df['garage_by_house_age'] = np.where(df['House.Age'] == 0, 0, df['Garage.Age']/df['House.Age'])

    
    df['garage_area_by_garage_cars'] =  df['Garage.Area']/df['Garage.Cars']

    df['garage_area_by_garage_cars'] = df['garage_area_by_garage_cars'].fillna(0)

    df['Has.Enclosed.Porch'] = (df['Enclosed.Porch'] > 0).astype(int)

    df['has_remodel'] = \
        df["Remod.Age"] == df["House.Age"]
    
    df['TotalSF'] = df['X1st.Flr.SF'] + df['X2nd.Flr.SF'] + df['Total.Bsmt.SF']

    # Total de Banheiros
    df['TotalBath'] = df['Full.Bath'] + 0.5 * df['Half.Bath'] + df['Bsmt.Full.Bath'] + 0.5 * df['Bsmt.Half.Bath']


    #Transforme age columns em categoricas

    df['House.Age'] = df['House.Age'].astype('category')   
    
    df['Yr.Sold'] = df['Yr.Sold'].astype('category')
    
    return df.drop(columns = ["Screen.Porch", "Enclosed.Porch", "Garage.Age", 'Garage.Cars', "Remod.Age", ])


Pipeline com Ridge

In [235]:
from sklearn.discriminant_analysis import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures,FunctionTransformer
from sklearn.compose import ColumnTransformer
columns_to_standardize = ['Lot.Area', 'X1st.Flr.SF', 'Bsmt.Unf.SF', 'Total.Bsmt.SF', 'Lot.Frontage']



preprocessor = ColumnTransformer(
    transformers=[
        ('std_scaler', StandardScaler(), columns_to_standardize),
        ('poly', PolynomialFeatures(degree=2, include_bias=False), colunas_numericas_poly)
    ],
    remainder='passthrough'
)

pipeRidge = Pipeline([
    ('teste', FunctionTransformer(teste)),
    ('preprocessor', preprocessor),
    ('cls', Ridge())
])

pipeRidge.fit(Xtrain,ytrain)
pred_pipeRidge = pipeRidge.predict(Xtest)


from sklearn.metrics import mean_squared_error

RMSE_pipeRidge = np.sqrt(mean_squared_error(ytest, pred_pipeRidge))

error_percent = 100 * (10**RMSE_pipeRidge - 1)
print(f'Average error for Ridge Pipeline is {error_percent:.2f}%')

KeyError: 'Garage.Area'

Pipeline e GridSearch com Ridge

In [ ]:
from sklearn.discriminant_analysis import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures,FunctionTransformer
from sklearn.compose import ColumnTransformer



columns_to_standardize = ['Lot.Area', 'X1st.Flr.SF', 'Bsmt.Unf.SF', 'Total.Bsmt.SF', 'Lot.Frontage']

preprocessor = ColumnTransformer(
    transformers=[
        ('std_scaler', StandardScaler(), columns_to_standardize),
        ('poly', PolynomialFeatures(degree=2, include_bias=False), colunas_numericas_poly)
    ],
    remainder='passthrough'
)

pipeRidge = Pipeline([
    ('teste', FunctionTransformer(teste)),
    ('preprocessor', preprocessor),
])



from sklearn.model_selection import GridSearchCV

# código utilizado: https://stackoverflow.com/questions/57376860/how-to-run-gridsearchcv-with-ridge-regression-in-sklearn

parameters = {'alpha':[0.0001,0.001,0.005,0.01,0.05,0.1,0.5,1,2,5,10,11,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,92,93,94,95,96,97,98,99,100]}

model = Ridge()

ridge_reg= GridSearchCV(model, parameters, scoring='neg_mean_squared_error',cv=10,n_jobs=-1)

pipeRidge.fit(Xtrain,ytrain)

Xtrain_mod = pipeRidge.transform(Xtrain)

ridge_reg.fit(Xtrain_mod,ytrain)

best_model = ridge_reg.best_estimator_

Xtest_mod = pipeRidge.transform(Xtest)

pred_pipeRidge = best_model.predict(Xtest_mod)

from sklearn.metrics import mean_squared_error

RMSE_pipeRidge = np.sqrt(mean_squared_error(ytest, pred_pipeRidge))

error_percent = 100 * (10**RMSE_pipeRidge - 1)
print(f'Average error for Ridge Pipeline is {error_percent:.2f}%')

Average error for Ridge Pipeline is 12.17%


c:\Users\code\Desktop\ML\BANGU2\ames-Bangu1\env\lib\site-packages\sklearn\linear_model\_ridge.py:200: LinAlgWarning: Ill-conditioned matrix (rcond=4.2207e-22): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


Pipeline com XGBRegressor

In [ ]:
from sklearn.discriminant_analysis import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.compose import ColumnTransformer
from xgboost import XGBRegressor



preprocessor = ColumnTransformer(
    transformers=[
        ('std_scaler', StandardScaler(), columns_to_standardize),
        ('poly', PolynomialFeatures(degree=2, include_bias=False), colunas_numericas_poly)
    ],
    remainder='passthrough'
)

pipeXGB = Pipeline([
    ('teste', FunctionTransformer(teste)),
    # ('preprocessor', preprocessor),
    ('cls', XGBRegressor())
])

pipeXGB.fit(Xtrain,ytrain)
pred_pipeXGB = pipeXGB.predict(Xtest)


from sklearn.metrics import mean_squared_error

RMSE_pipeXGB = np.sqrt(mean_squared_error(ytest, pred_pipeXGB))

error_percent = 100 * (10**RMSE_pipeXGB - 1)
print(f'Average error for XGBoost Pipeline is {error_percent:.2f}%')


c:\Users\code\Desktop\ML\BANGU2\ames-Bangu1\env\lib\site-packages\xgboost\data.py:463: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  or is_sparse(dtype)
c:\Users\code\Desktop\ML\BANGU2\ames-Bangu1\env\lib\site-packages\xgboost\data.py:464: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  or (is_categorical_dtype(dtype) and enable_categorical)


ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, The experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:Yr.Sold: category, House.Age: category

Pipeline com CatBoost

In [ ]:
from catboost import CatBoostRegressor
from sklearn.discriminant_analysis import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.compose import ColumnTransformer



preprocessor = ColumnTransformer(
    transformers=[
        ('std_scaler', StandardScaler(), columns_to_standardize),
        ('poly', PolynomialFeatures(degree=3, include_bias=False), colunas_numericas_poly)
    ],
    remainder='passthrough'
)

pipeCatBoost = Pipeline([
    ('teste', FunctionTransformer(teste)),
    ('preprocessor', preprocessor),
    ('cls', CatBoostRegressor(iterations=10000, 
                          depth=4, 
                          learning_rate=0.01, 
                          loss_function='RMSE'))
    # ('cls', CatBoostRegressor(iterations=10000, 
    #                       depth=6, 
    #                       learning_rate=0.01, 
    #                       loss_function='RMSE'))
])

pipeCatBoost.fit(Xtrain,ytrain)
pred_pipeCatBoost = pipeCatBoost.predict(Xtest)


from sklearn.metrics import mean_squared_error

RMSE_pipeCatBoost = np.sqrt(mean_squared_error(ytest, pred_pipeCatBoost))

error_percent = 100 * (10**RMSE_pipeCatBoost - 1)
print(f'Average error for CatBoost Pipeline is {error_percent:.2f}%')


0:	learn: 0.1682370	total: 161ms	remaining: 26m 53s
1:	learn: 0.1671653	total: 172ms	remaining: 14m 19s
2:	learn: 0.1660829	total: 183ms	remaining: 10m 9s
3:	learn: 0.1649821	total: 195ms	remaining: 8m 6s
4:	learn: 0.1639121	total: 206ms	remaining: 6m 51s
5:	learn: 0.1628648	total: 219ms	remaining: 6m 4s
6:	learn: 0.1618762	total: 230ms	remaining: 5m 28s
7:	learn: 0.1607861	total: 241ms	remaining: 5m 1s
8:	learn: 0.1598083	total: 252ms	remaining: 4m 39s
9:	learn: 0.1587885	total: 264ms	remaining: 4m 23s
10:	learn: 0.1577625	total: 276ms	remaining: 4m 10s
11:	learn: 0.1567238	total: 287ms	remaining: 3m 58s
12:	learn: 0.1556695	total: 298ms	remaining: 3m 49s
13:	learn: 0.1546638	total: 310ms	remaining: 3m 41s
14:	learn: 0.1536880	total: 321ms	remaining: 3m 33s
15:	learn: 0.1526814	total: 334ms	remaining: 3m 28s
16:	learn: 0.1516900	total: 344ms	remaining: 3m 22s
17:	learn: 0.1507538	total: 356ms	remaining: 3m 17s
18:	learn: 0.1497678	total: 367ms	remaining: 3m 12s
19:	learn: 0.1488243	to